Εκφώνηση Ερωτήματος 1
******
Το εργοστάσιο ῾῾Ουράνιο Τόξο᾿᾿, μεταξύ των διαφόρων χρωμάτων και σχετικών προϊόντων που παράγει,
παράγει και πέντε διαφορετικές ειδικές αποχρώσεις χρωμάτων (με ονομασίες: 1, 2, 3, 4 και 5) για
μεγάλους πελάτες της, για τις οποίες η ζήτηση είναι σταθερή. Κάθε τέτοια ειδική απόχρωση προκύπτει
από ένα συγκεκριμένο μηχάνημα ανάμιξης το οποίο θα πρέπει να καθαρίζεται πριν την παραγωγή της
επόμενης (διαφορετικής) απόχρωσης. Η διαδικασία ανάμιξης για κάθε απόχρωση από το 1 έως το 5
διαρκεί 40, 35, 45, 32 και 50 λεπτά, αντίστοιχα. Οι χρόνοι καθαρισμού του παραπάνω μηχανήματος
εξαρτώνται τόσο από τον τύπο όσο και από την απόχρωση των χρωμάτων. Για παράδειγμα, ένα χρώμα
που είναι βασισμένο σε λάδι απαιτεί παραπάνω χρόνο καθαρισμού απο ένα χρώμα που έχει βάση του το
νερό. Κάτι αντίστοιχο συμβαίνει όταν μετά απο μια απόχρωση σκούρου χρώματος θα πρέπει να παραχθεί
μια απόχρωση του λευκού. Ο χρόνος καθαρισμού του μηχανήματος (σε λεπτά) που απαιτούνται ώστε το
μηχάνημα να παράξει την απόχρωση j δεδομένου ότι η αμέσως προηγούμενη απόχρωση ήταν η i δίνεται
από το στοιχείο (i, j) του παρακάτω πίνακα.


Επειδή το εργοστάσιο έχει και άλλες δραστηριότητες παραγωγής, η διοίκησή του επιθυμεί η εβδομαδιαία παραγωγή των πέντε ειδικών αποχρώσεων να γίνεται σε όσο το δυνατόν μικρότερο χρόνο (αναμίξεις
και καθαρισμοί). Ποια σειρά παραγωγής των αποχρώσεων εξυπηρετεί βέλτιστα την παραπάνω επιθυμία
της διοίκησης; Η σειρά θα επαναλαμβάνεται κάθε εβδομάδα, επομένως ο χρόνος καθαρισμού μεταξύ της
τελευταίας παραγωγής απόχρωσης μιας εβδομάδας και της πρώτης παραγωγής απόχρωσης της επόμενης
εβδομάδας θα πρέπει να προσμετρηθεί στον συνολικό χρόνο καθαρισμού του μηχανήματος.

In [29]:
import numpy as np

from pyomo.environ import *
from pyomo.gdp import *

from ortools.constraint_solver import pywrapcp, routing_enums_pb2

Θέλουμε την παραγωγή των 5 χρωμάτων με τον ελάχιστο δυνατό χρόνο ανά εβδομάδα. Επειδή οι χρόνοι για ανάμειξη είναι δεδομένοι και σταθεροί, μπορούμε να τους προσθέσουμε απλά στο τελικό αποτέλεσμα. Άρα ψάχνουμε τον βέλτιστο τρόπο να περάσουμε από όλα τα χρώματα (κόμβους) και να επιστρέψουμε πάλι στον αρχικό (για να είναι έτοιμο το μηχάνημα για την επόμενη εβδομάδα). Το προβλημα μοιάζει αρκετά με το πρόβλημα του πλανόδιου πωλητή, ή ένα πρόβλημα διάσχισης γράφου, για αυτό και χρησιμοποιώ τον συγκεκριμένο τρόπο επίλυσης.

In [30]:
#Εισάγω τα δεδομένα για το πρόβλημα (τους χρόνους καθαρισμού και αναμείξεων)

def create_data_model():
    
    data = {}
    data['clean_times'] =  [
        [0, 11, 7, 13, 11],
        [5, 0, 13, 15, 15],
        [13, 15, 0, 23, 11],
        [9, 13, 5, 0, 3],
        [3, 7, 7, 7, 0]
    ]
    data['num_machines'] = 1
    data['depot'] = 0
    data['mix_times'] = [40, 35, 45, 32, 50]
    
    return data

In [31]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['clean_times']),
                                       data['num_machines'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [32]:
#Βρίσκω τον χρόνο εναλλαγής δύο χρωμάτων, δηλαδή την 'απόσταση' μεταξύ δύο κελιών. 
def distance_callback(from_index, to_index):

    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['clean_times'][from_node][to_node]


Ορίζω ένα έτοιμο μοντέλο της pywrapcp βιβλιοθήκης το οποίο λύνει προβλήματα, όπως το πρόβλημα του πλανόδιου πωλητή. Το μοντέλο χρησιμοποιεί τη στρατηγική PATH_CHEAPEST_ARC, δηλαδή ξεκινάει από έναν αρχικό κόμβο και προσθέτει ακμές με το μικρότερο βάρος, που δεν οδηγούν σε κόμβο που έχει ήδη επισκεφθεί.

In [33]:
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [34]:
def print_solution(manager, routing, solution):

    incompressible_time = np.sum(data['mix_times'])  
    print('Minimum time needed: {} minutes'.format(solution.ObjectiveValue() + incompressible_time))
    
    index = routing.Start(0)
    plan_output = 'Optimal paint production:\n'
    cleaning_times = 0

    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index)+ 1)
        previous_index = index
        index = solution.Value(routing.NextVar(index)) 
        cleaning_times += routing.GetArcCostForVehicle(previous_index, index, 0)

    plan_output += ' {}\n'.format(manager.IndexToNode(index)+ 1)

    print(plan_output)
    plan_output += 'Optimal paint production: {}minutes\n'.format(cleaning_times)
    

In [35]:
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print_solution(manager, routing, solution)

Minimum time needed: 243 minutes
Optimal paint production:
 1 -> 4 -> 3 -> 5 -> 2 -> 1

